# Python Tutorial for Cell Segmentation and smFISH
### Modified from original code from the Q-Bio Summer School

```
https://colab.research.google.com/drive/1o3JJE4EjfW9P5ZITEeS5p8Ui60xV3etK?usp=sharing

Copyright (c) 2022 Dr. Brian Munsky. 
Dr. Luis Aguilera, Will Raymond
Colorado State University.
Licensed under BSD-3-Clause license.
```



# Abstract 

In this notebook, we will talk about single-cell segmentation using Python. Basic image manipulation in Python can be reviewed here:  [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1cOzLyKrKznlc2olymshHeQMalCG9IydW?usp=sharing).  

In this tutorial, our goal is introduce the basics of single-cell segmentation. 

## List of objectives

1. Understand and explain the more common methods used to segment cells from microscope images.
2. Understand and explain what a segmentation mask is.
3. Understand and explain segmentation methods based on threshold selection.
4. Perform single-cell segmentation using machine learning based methods. 


<img src= https://github.com/MunskyGroup/uqbio2022/raw/master/files/files_image_processing/module_1_2/images/Slide2.png alt="drawing" width="1200"/>

<img src= https://github.com/MunskyGroup/uqbio2022/raw/master/files/files_image_processing/module_1_2/images/Slide3.png alt="drawing" width="1200"/>

<img src= https://github.com/MunskyGroup/uqbio2022/raw/master/files/files_image_processing/module_1_2/images/Slide4.png alt="drawing" width="1200"/>

### Do it by hand
Using software such as [ImageJ/FIJI](https://imagej.nih.gov/ij/), [Napari](https://napari.org) or even something like Microsoft Paint, one can manually outline cells. This is cumbersome and impractical for processing thousands of cells over time. 

<img src= https://github.com/MunskyGroup/uqbio2022/raw/master/files/files_image_processing/module_1_2/images/Slide5.png alt="drawing" width="1200"/>



Check this tool ([makesense](https://www.makesense.ai)) to create your own masks.




You can find some images in the following [link](https://www.dropbox.com/s/d9my4cp2j3ven04/test_data_uqbio2022.zip?dl=0)

# Getting started with segmentation using thresholding


### Watershed Methods
The scikit-image library has an excellent tutorial on [watershed methods](https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_watershed.html). Popular tools that apply such methods to single cells are:
* [CellStar](http://cellstar-algorithm.org) (Matlab, Python, CellProfiler PlugIn)
* [FogBank](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-014-0431-x#additional-information) (Matlab)

<img src= https://github.com/MunskyGroup/uqbio2022/raw/master/files/files_image_processing/module_1_2/images/Slide6.png alt="drawing" width="1200"/>

In [ ]:
#%%capture
# Loading libraries
import random                        # Library to generate random numbers
import skimage                       # Library for image manipulation
import numpy as np                   # Library for array manipulation
import urllib.request                # Library to download data
import matplotlib.pyplot as plt      # Library used for plotting
from skimage import io               # Module from skimage
from skimage.io import imread        # Module from skimage to read images as numpy arrays
from skimage.filters import gaussian # Module working with a gaussian filter
import os                            # Module for file manipulation on different operating systems
%matplotlib inline

In [ ]:
import matplotlib
matplotlib.__version__

Let's get started by downloading a sample image of a cell and plotting it using `matplotlib`:


In [ ]:
# Downloading a test image
if not os.path.exists('./image_cell.tif'):
    !wget -O image_cell.tif https://ndownloader.figshare.com/files/26751209

# Loading figure to the notebook
images = imread('./image_cell.tif') 
print('File is accessible as ./image_cell.tif')


In [ ]:
# Explain        figName = './image_cell.tif'   what is our cwd?    #import os #os.getcwd()
#random.choice(participants)

In [ ]:
# Printing the shape of the image
print('Original image shape: ' , images.shape)  # [T,Y,X,C]
# Selecting a frame and a color channel
img = images[0,:,:,0]
print('Single image shape: ' , img.shape)  # [Y,X]

In [ ]:
# Difference  between images and img?
#random.choice(participants)

In [ ]:
# Plotting the image as the 3d dimension figure.
space= np.arange(0, img.shape[0], 1)
xx, yy = np.meshgrid(space,space)
fig = plt.figure(figsize=(15,7))
# Set up the axes for the first plot
ax = fig.add_subplot(1, 2, 1)
ax.imshow(img,cmap='Spectral') # Reds_r
# Set up the axes for the second plot
ax2 = fig.add_subplot(1, 2, 2, projection='3d')
ax2.plot_surface(xx, yy , img,  rstride=20, cstride=20, shade=False, cmap='Spectral')
ax2.view_init(20, 45)
plt.show()

Recall when we plotted the histogram of the intensity pixels to get a sense of the distribution of pixel intensities throughout the image:

In [ ]:
# Plotting the intensity distribution
f, ax = plt.subplots()
_ = ax.hist(img.ravel(),color='orangered',bins=35)
ax.set_xlabel('pixel')
ax.set_ylabel('# of pixels')
plt.show()

In [ ]:
# _ = ax.hist(img.ravel(),color='orangered',bins=35)
#random.choice(participants)

Based on this image, we can guess a threshold of pixel intensities that are "cells" vs "not cells". What do you think would make a good threshold?

_Modify the cell below and try different thresholds_


In [ ]:
# Thresholding the image
threshold = 700   # Please play  with this threshold
mask_image = np.zeros(img.shape)
mask_image[img>threshold] = 255
f,ax = plt.subplots()
ax.imshow(mask_image, cmap='Greys')
plt.show()

This mask image is useful, especially considering we simply took all of the pixels with a value bigger than `threshold`. 

However, we know that the outside ought to be more smooth. Let's try applying a Gaussian filter to smooth out the mask image:


In [ ]:
# Importing library with the watershed algorithm. 
from skimage.morphology import binary_dilation, watershed

In [ ]:
# Applying a gaussian filter to the image
new_mask = gaussian(mask_image, sigma=5) 
f,ax = plt.subplots()
ax.imshow(new_mask, cmap='Greys')
plt.plot()

In [ ]:
# Importing a library to find contours in the image
from skimage import measure

In [ ]:
# Plotting all contours detected in the filtered image
f,ax = plt.subplots()
contours = measure.find_contours(new_mask, level=125 ) # level is half of 255 (ish). What happens if we change it?
ax.imshow(new_mask, cmap='Greys')
for contour in contours:
  ax.plot(contour[:,1],contour[:,0],color='r')

In [ ]:
#help(measure.find_contours)

In [ ]:
# Plotting the countour on top of the original image
img = images[0,:,:,0]
f,ax = plt.subplots()
ax.imshow(img, cmap='Greys')
for contour in contours:
  ax.plot(contour[:,1],contour[:,0],'r')

So far so good. By setting `threshold=700` we were able to find the "main" cell in the image. But what happens when we want to get all three? Let's start by lowering the threshold to  300 and running the code. 

In [ ]:
# Thresholding with a lower value
threshold = 300
mask_image = np.zeros(img.shape)
mask_image[img>threshold] = 255
new_mask = gaussian(mask_image, sigma=4) # applaying the gaussian filter
contours = measure.find_contours(new_mask, level=125, fully_connected='high') # Finding the contours in the image
img = images[0,:,:,0]

#  Plotting the  contour detected on top of the original image
f,ax = plt.subplots()
ax.imshow(img, cmap='Spectral')
contours_connected = np.vstack((contours))
print(contours_connected.shape)
for contour in contours:
  ax.plot(contour[:,1],contour[:,0],'-b',lw=8)

# Connecting the last and first  elements in the array (contours) to get a fully connected shape
contours_connected = np.vstack((contours_connected[-1,:],contours_connected))
print(contours_connected.shape)

# Plotting
ax.plot(contours_connected[:,1],contours_connected[:,0],'y',lw=3)
plt.show()

_it looks like a crab_ !


In the cell below, we will try to take the connected image below and use a [watershed algorithm](https://en.wikipedia.org/wiki/Watershed_(image_processing) to break it into 3 distinct cells. 

In [ ]:
# importing a library to convert contours into shapes.
from skimage.draw import polygon

In [ ]:
# make a new mask from the contours array
watershed_starting_mask = np.zeros(img.shape).astype(int)                    # Prealocating an array with zeros. Notice the datatype.
rr, cc = polygon(contours_connected[:,0], contours_connected[:,1])   # Returns the coordinates inside the contour
watershed_starting_mask[rr,cc] = 1                                           # Replacing all values inside the contour with ones.

# Plotting the mask 
f,ax = plt.subplots()
ax.imshow(watershed_starting_mask, cmap='Greys_r')
plt.show()

# Printing the minimum and maximum values in the image
print('min value in mask: ', np.min(watershed_starting_mask) )
print('max value in mask: ', np.max(watershed_starting_mask) )

In [ ]:
#watershed_starting_mask = np.zeros(img.shape).astype(int)                    # Prealocating an array with zeros. Notice the datatype.
#random.choice(participants)

In [ ]:
# Importing libraries with the watershed algorithm and local maximum detection
from scipy import ndimage as ndi              # Distance Transform
from skimage.feature import peak_local_max    # Local maxima in a matrix
from skimage.segmentation import watershed    # Watershed algorithm

To find more information about the specific method use

```
help(watershed)
```



### Distance transform



"The distance transform computes the distance between each pixel and the nearest zero/nonzero pixel." An example with code implementation is accessible in this [link](https://www.youtube.com/watch?v=oxWfLTQoC5A).

For more infromation about the distance transform check this [link](https://homepages.inf.ed.ac.uk/rbf/HIPR2/distance.htm)

<img src= https://homepages.inf.ed.ac.uk/rbf/HIPR2/figs/distance.gif alt="drawing" width="600"/>



By  using the distance transform we can find basins in the center of each cell.

In [ ]:
# Computes the Distance Transform distance in the image
distance = -ndi.distance_transform_edt(watershed_starting_mask)                       

# Plotting the image as the 3d dimension figure.
space= np.arange(0, distance.shape[0], 1)
xx, yy = np.meshgrid(space,space)
fig = plt.figure(figsize=(15,7))
# Set up the axes for the first plot
ax = fig.add_subplot(1, 2, 1)
ax.imshow(distance,cmap='Spectral') # Reds_r
# Set up the axes for the second plot
ax2 = fig.add_subplot(1, 2, 2, projection='3d')
ax2.plot_surface(xx, yy , distance,  rstride=5, cstride=5, shade=False, cmap='Spectral')
#ax2.view_init(30, 45)
plt.show()

In [ ]:
# Apply watershed
distance = ndi.distance_transform_edt(watershed_starting_mask)                       # Computes the Distance Transform distance in the image
coords = peak_local_max(distance, min_distance=50, labels=watershed_starting_mask)   # Use the Distance transform image to find local maxima
_,inds = np.unique(distance[coords[:,0],coords[:,1]],return_index=True)      # Make sure they are unique
coords = coords[inds,:]                                                      # Selecting unique indexes
mask = np.zeros(distance.shape, dtype=bool)                                  # Prealocating an array with zeros
mask[tuple(coords.T)] = True                                                 # Make an image with 1's where local maxima are
markers, _ = ndi.label(mask)                                                 # Unique values used as the desired labels

# Using the watershed algorithm
labels = watershed(-distance, markers, mask=watershed_starting_mask, watershed_line=True)  #Why do we need to use the negative of the distance matrix?

# Plotting the results
f,ax = plt.subplots(1,5, figsize=(15,7))
ax[0].imshow(img, cmap='Spectral')
ax[0].set_title('origninal')
ax[1].imshow(watershed_starting_mask, cmap='Greys_r')
ax[1].set_title('Mask')
ax[2].imshow(ndi.distance_transform_edt(watershed_starting_mask), cmap='Greys')
ax[2].set_title('Distance Transform')
ax[3].imshow(ndi.distance_transform_edt(watershed_starting_mask), cmap='Greys')
ax[3].scatter(coords[:,1],coords[:,0],c='r')
ax[3].set_title('Local Maxima in Dist. Transform')
ax[4].imshow(labels, cmap='Spectral')
ax[4].set_title('Masks with Labels')
f.tight_layout() 

In [ ]:
np.unique(labels)

In [ ]:
#plt.imshow(labels[100:300,100:300])

In [ ]:
# Why do we need to use the negative of the distance matrix?
#random.choice(participants)

In [ ]:
#help(watershed)

# Machine Learning Methods





* Please notice that a complete tutorial on Machine Learning will be given on June 7 by Will Raymond.


In recent years, deep learning methods have rapidly improved the state of the art for cell segmentation methods. We will come back to the theory on this topic - for now, we will demonstrate a couple of ML-based tools that can be used to segment images. If you are keen to get started learning about how the popular U-Net model works, check out [this video](https://www.youtube.com/watch?v=azM57JuQpQI) and/or [this video](https://www.youtube.com/watch?v=4ZZjr6SFBV8).


<img src= https://github.com/MunskyGroup/uqbio2022/raw/master/files/files_image_processing/module_1_2/images/Slide7.png alt="drawing" width="1200"/>

<img src= https://github.com/MunskyGroup/uqbio2022/raw/master/files/files_image_processing/module_1_2/images/Slide8.png alt="drawing" width="1200"/>

[Unet code implementation](https://github.com/zhixuhao/unet).

## Segment yeast brightfield images using U-Net

In [ ]:
%%capture
# Downloading and cloning micromator repository
%cd gdrive/MyDrive/
! git clone https://gitlab.inria.fr/InBio/Public/micromator.git
! pip install pims
! pip install trackpy
%cd micromator

In [ ]:
# Explain the previous code
#random.choice(participants)

In [ ]:
%%capture
# Importing modules from micromator
from micromator.segmator import seg_mator

In [ ]:
# Downloading a test image
# yeast_image = sk.io.imread('../drive/MyDrive/binned_img_51_BF.tif')
urls = ['https://github.com/MunskyGroup/uqbio2021/raw/main/module_1/images/binned_img_65_BF.tif']
print('Downloading file...')
urllib.request.urlretrieve(urls[0],'./binned_img_65_BF.tif')

In [ ]:
# Loading the brightfield image
yeast_image = imread('./binned_img_65_BF.tif')
print(yeast_image.shape)

In [ ]:
# Plotting the brightfield image
f, ax = plt.subplots() 
ax.imshow(yeast_image[600:800,50:250], cmap='Greys')
plt.show()

In [ ]:
%%capture
# Initializing micromator
sm = seg_mator.SegMator('', '', 'segmator/models/unet_yeast_seg_new_v3.hdf5', '.',input_size=(1024,1024,1))

In [ ]:
#help(seg_mator.SegMator)

In [ ]:
# Applying the unet method to our image
image, mask, contours = sm.segment_single_frame(yeast_image, target_size=(1024,1024))

In [ ]:
# Explain the outputs ----> image, mask, contours 
#random.choice(participants)

In [ ]:
# Plotting the original image and mask
f,ax = plt.subplots(1,2)
ax[0].imshow(image[600:800,50:250],cmap='Greys')
ax[1].imshow(mask[600:800,50:250],cmap='Greys')
plt.show()

In [ ]:
# Plotting a single contour
f,ax = plt.subplots()
ax.plot(contours[5][:,1],contours[5][:,0])
plt.show()

In [ ]:
# Plotting a mask for a single cell
contour = contours[300]
single_cell_mask = np.zeros(image.shape)
rr, cc = polygon(contour[:,0], contour[:,1])
single_cell_mask[rr,cc] = 1
f,ax = plt.subplots()
ax.imshow(single_cell_mask,cmap='Greys')
cell_size = single_cell_mask.sum()
print(cell_size)

In [ ]:
# Plotting the contours for all cells
f,ax = plt.subplots()
ax.imshow(image, cmap='Greys')
for contour in contours:
  ax.plot(contour[:,1],contour[:,0],'r',alpha=.5)
ax.set_xlim([0,200])
ax.set_ylim([50,250])

In [ ]:
def get_cell_size(contour, image):
  '''
  This function is intended to calculate cell size from a contour.
  '''
  single_cell_mask = np.zeros(image.shape)
  rr, cc = polygon(contour[:,0], contour[:,1])
  single_cell_mask[rr,cc] = 1
  cell_size = single_cell_mask.sum()
  return cell_size

In [ ]:
# Making a list with the cell size for each cell.
all_cell_sizes = []
for contour in contours:
  all_cell_sizes.append(get_cell_size(contour, image))

In [ ]:
# Plotting an histogram with the cell size
_ = plt.hist(all_cell_sizes, bins=40,color=[0.5,0.5,0.5] )
plt.xlabel('cell size (pixels)')
plt.ylabel('# of cells')
plt.show()

#Cellpose

The [CellPose](https://www.nature.com/articles/s41592-020-01018-x) algorithm uses a [U-Net approach](https://arxiv.org/pdf/1505.04597.pdf), but is a generalist algorithm that can work with a wide variety of cell types. Published in 2021.  ~ 360 citations.

<img src= https://github.com/MunskyGroup/uqbio2022/raw/master/files/files_image_processing/module_1_2/images/Slide9.png alt="drawing" width="1200"/>

<img src= https://github.com/MunskyGroup/uqbio2022/raw/master/files/files_image_processing/module_1_2/images/Slide10.png alt="drawing" width="1200"/>

One of the biggest problems in single-cell segmentation is the limited number of images that are needed to traing a machine learning algorithm.

<img src= https://github.com/MunskyGroup/uqbio2022/raw/master/files/files_image_processing/module_1_2/images/Slide11.png alt="drawing" width="1200"/>

<img src= https://github.com/MunskyGroup/uqbio2022/raw/master/files/files_image_processing/module_1_2/images/Slide12.png alt="drawing" width="1200"/>

### Segmenting a complete cell using Cellpose

In [ ]:
# Downloading a test image
# Hela Cells. Linda's Publication.
urls = ['https://ndownloader.figshare.com/files/26751209']
urllib.request.urlretrieve(urls[0], './image_cell.tif')
figName = './image_cell.tif'
image_complete = imread(figName) 

In [ ]:
print('image shape: ', image_complete.shape)

In [ ]:
# Plotting each one of the 3 colors independently
fig, ax = plt.subplots(1,3, figsize=(20, 7))
ax[0].imshow(image_complete[0,:,:,0],cmap='Reds_r')
ax[1].imshow(image_complete[0,:,:,1],cmap='Greens_r')
ax[2].imshow(image_complete[0,:,:,2],cmap='Blues_r')
ax[0].axis('off')
ax[1].axis('off')
ax[2].axis('off')
plt.show()

Installing cellpose

In [ ]:
%%capture
!pip install opencv-python-headless==4.1.2.30
!pip install cellpose==1.0
from cellpose import models
from cellpose import plot

In [ ]:
img= image_complete[0,:,:,0]

In [ ]:
# RUN CELLPOSE
use_GPU = True # models.use_gpu()
# DEFINE CELLPOSE MODEL
model = models.Cellpose(gpu=use_GPU, model_type='cyto') # model_type='cyto' or model_type='nuclei'
# Running the models
masks, flows, styles, diams = model.eval(img, diameter=200, flow_threshold=None, channels=[0,0],omni=True)
plt.imshow(masks,cmap='Greys')
plt.show()

In [ ]:
#help(model.eval)

In [ ]:
# Downloading a test image
# Hela Cells. Linda's Publication.
urls = ['https://ndownloader.figshare.com/files/26751203']
urllib.request.urlretrieve(urls[0], './image_cell.tif')
figName = './image_cell.tif'
image_complete = imread(figName) 

In [ ]:
img= image_complete[0,:,:,2]

In [ ]:
# RUN CELLPOSE
use_GPU = True # models.use_gpu()
# DEFINE CELLPOSE MODEL
model = models.Cellpose(gpu=use_GPU, model_type='cyto') # model_type='cyto' or model_type='nuclei'
# Running the models
masks, flows, styles, diams = model.eval(img, diameter=200, flow_threshold=None, channels=[0,0],omni=True)
plt.imshow(masks,cmap='Greys')
plt.show()

### Segmenting nucleus and cytosol

In [ ]:
# Downloading a FISH image
urls = ['https://github.com/MunskyGroup/FISH_Processing/raw/main/dataBases/test_data/ROI001_XY1620755243_Z00_T0_merged.tif']
print('Downloading file...')
urllib.request.urlretrieve(urls[0], './ROI001_XY1620755243_Z00_T0_merged.tif')
figName = './ROI001_XY1620755243_Z00_T0_merged.tif'
images_FISH = imread(figName) 
print('File Downloaded!')

In [ ]:
# The image has the following dimensions [Z,Y,X,C]
print(images_FISH.shape)

In [ ]:
# For segmentation, we will select the central  slice.
image_to_segment= images_FISH[10,:,:,:]

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(20, 10))
ax[0].imshow(images_FISH[10,:,:,0],cmap='Spectral_r')
ax[0].set(title='Ch0 - DAPI')
ax[1].imshow(images_FISH[10,:,:,1],cmap='Spectral_r')
ax[1].set(title= 'Ch1 - FISH vs MS2  reporter gene' )
ax[2].imshow(images_FISH[10,:,:,2],cmap='Spectral_r')
ax[2].set(title= 'Ch1 - FISH vs GAPDH' )
plt.show()

In [ ]:
# Segmenting the nucleus
img_nuc = images_FISH[10,:,:,0:2]
print(img_nuc.shape)

To get information about the parameters needed by the library


```
help(model.eval)
```

```
help(models.Cellpose)
```

In [ ]:
# RUN CELLPOSE
use_GPU = True # models.use_gpu()
# DEFINE CELLPOSE MODEL
model = models.Cellpose(gpu=use_GPU, model_type='nuclei') # model_type='cyto' or model_type='nuclei'
# Running the models
masks_nuc, flows, styles, diams = model.eval(img_nuc, diameter=100, flow_threshold=None, channels=[0,1], net_avg=True, augment=True)

# Plotting the results
plt.imshow(masks_nuc,cmap='Greys_r')
plt.show()

In [ ]:
# Segmenting the nucleus
img_cyto = images_FISH[10,:,:,0:3]
print(img_cyto.shape)

In [ ]:
# RUN CELLPOSE
use_GPU = True # models.use_gpu()
# DEFINE CELLPOSE MODEL
model = models.Cellpose(gpu=use_GPU, model_type='cyto2') # model_type='cyto', 'cyto2' or model_type='nuclei'
# Running the models
masks_cyto, flows, styles, diams = model.eval(img_cyto, diameter=200, flow_threshold=None, channels=[0,2], net_avg=True, augment=True)

# Plotting the results
plt.imshow(masks_cyto,cmap='Greys_r')
plt.show()

## Calculating the [center of mass](https://www.khanacademy.org/science/physics/linear-momentum/center-of-mass/a/what-is-center-of-mass) for a given cell assuming that the cell is a 2D object.  Assuming  $A = M$.

For a distribution of mass:

$x_{CM} = \int\limits_0^M \frac{x \,dm}{M}$  ,   
$y_{CM} = \int\limits_0^M \frac{y \,dm}{M}$  ,



where $M$ is the total mass in the system.


In [ ]:
# Calculaitng  the center of mass for a selected cell.
selected_mask_cyto = np.where(masks_cyto==1,1,0) # Selecting only one mask.    
y_indexes = np.nonzero(selected_mask_cyto)[0] # Detecting the indexes for all x axis
x_indexes = np.nonzero(selected_mask_cyto)[1] # Detecting the indexes for all y axis

center_mass_x = int ( np.sum(x_indexes) / np.sum(selected_mask_cyto) )
center_mass_y = int( np.sum(y_indexes) / np.sum(selected_mask_cyto) )

print('The center of mass is located in y: ' , center_mass_y,'x: ' ,  center_mass_x) 

# Plotting the selected cell and its center of mass
plt.imshow(selected_mask_cyto,cmap='Greys_r')
plt.plot(center_mass_x,center_mass_y,'r+',markersize=12)
plt.show()

# Practice Problems

## Single Cell Segmentation - Workbook Completion Requirements:##
To obtain credit for this lesson, each student should complete all blanks for questions Q1-Q6.

To obtain a certificate for the course, you must complete a minimum of five notebooks from Modules 1-4 (please note that preliminary notebooks from Module 0 will not be accepted) and submit them together via email before August 15, 2022. Please submit your completed notebooks to qbio_summer_school@colostate.edu

## Easy Questions

Using the video accessible in this link 'https://ndownloader.figshare.com/files/26751209', solve the following questions:



* Q1. Plot the average intensity in the cell for all time points.


In [ ]:
# Downloading a test image
urls = ['https://ndownloader.figshare.com/files/26751209']
print('Downloading file...')
figName = './image_cell.tif'
urllib.request.urlretrieve(urls[0], figName)
# Loading figure to the notebook
imgs_2 = imread(figName) 
print('File is downloaded and accessible in: ... /contents/image_cell.tif ')

In [ ]:
print(imgs_2.shape)

In [ ]:
# Need a mask for the cell -- tels us which pixels are inside the cell.
# RUN CELLPOSE
use_GPU = True # models.use_gpu()
# DEFINE CELLPOSE MODEL
model = models.Cellpose(gpu=use_GPU, model_type='cyto') # model_type='cyto' or model_type='nuclei'
# Running the models

masks, flows, styles, diams = model.eval(imgs_2[0,:,:,:], diameter=200, min_size=1000, channels=[0,0])
plt.imshow(masks,cmap='Greys')
plt.show()

# Make a list with all the masks separated out
masks_list = []
for i in range(np.max(masks)+1):
  masks_list.append(masks==i)

# Plot a particular mask
plt.imshow(masks_list[2],cmap='Greys')
plt.show()

# Plot an image but only showing the part in the mask
plt.imshow(imgs_2[0,:,:,0]*masks_list[2],cmap='Greys')
plt.show()

# Compute and plot the mean intensities over time of for each color and each cell.
fig, ax = plt.subplots(1,3, figsize=(20, 5))
for k in range(3): 
  matrix_intensity_in_time = np.zeros((3,imgs_2.shape[0]))
  for i in range(imgs_2.shape[0]):
    for j in range(3):
      matrix_intensity_in_time[j,i] = np.sum(imgs_2[i,:,:,j]*masks_list[k+1])/np.sum(masks_list[k+1])
  ax[k].set_prop_cycle(color=['red', 'green', 'blue'])
  ax[k].plot(matrix_intensity_in_time.T/matrix_intensity_in_time[:,0])
  plt.legend(['red','green','blue'])

* Q2. Calculate the pixel intensity distribution inside the cell for  time point 0.


In [ ]:
# Add your code with your response here:

# Applying the mask to original image
selected_frame = 0 
selected_channel =0 
image_with_mask = imgs_2[selected_frame,:,:,selected_channel]*masks_list[2]
plt.imshow(image_with_mask)

# Selecting non-zero pixels. The pixeles inside the mask.
non_zero_pixels = image_with_mask[image_with_mask>0]
non_zero_pixels.shape

# Plotting the intensity distribution for a specific time point and an specific channel
plt.figure(figsize=(7,7))
plt.hist(non_zero_pixels, bins=80,color='orangered')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Intensity Histogram')
plt.show()


* Q3. Calculate a mask for every frame (time point) in the video.

In [ ]:
# Add your code with your response here:

# Need a mask for the cell -- tels us which pixels are inside the cell.
# RUN CELLPOSE
use_GPU = True # models.use_gpu()
# DEFINE CELLPOSE MODEL
model = models.Cellpose(gpu=use_GPU, model_type='cyto') # model_type='cyto' or model_type='nuclei'

# Running the model for all  frames
masks_for_all_frames, _, _, _ = model.eval(imgs_2[:,:,:,:], diameter=200, min_size=1000, channels=[0,0])

# Notice that the output is a tensor with  shape [T,Y,X]
print(masks_for_all_frames.shape)

# Plotting the  masks founds for every frame.
number_frames= imgs_2.shape[0]
fig, ax = plt.subplots(nrows=1,ncols=number_frames, figsize=(20, 5))
for i in range(number_frames):
  ax[i].imshow(masks_for_all_frames[i,:,:])
  ax[i].axis('off')

## Moderate Questions

Using the FISH image accessible in this link 'https://github.com/MunskyGroup/FISH_Processing/raw/main/dataBases/test_data/ROI002_XY1620755646_Z00_T0_merged.tif', solve the following questions:

In [ ]:
# Downloading a FISH image
urls = ['https://github.com/MunskyGroup/FISH_Processing/raw/main/dataBases/test_data/ROI002_XY1620755646_Z00_T0_merged.tif']
print('Downloading file...')
urllib.request.urlretrieve(urls[0], './ROI001_XY1620755243_Z00_T0_merged.tif')
figName = './ROI001_XY1620755243_Z00_T0_merged.tif'
images_FISH = imread(figName) 
print('File Downloaded!')

* Q4. Using a maximum projection in Z, calculate the mask for the nuclei and cytosol in the FISH image.

In [ ]:
# Add your code with your response here:
print('The shape of the FISH image is : ', images_FISH.shape)  #[Z,Y,X,C]

# Inspecting the image to determine the channel with nucleus and cytosol
fig, ax = plt.subplots(1,3, figsize=(20, 10))
ax[0].imshow(images_FISH[10,:,:,0],cmap='Spectral_r')
ax[0].set(title='Ch0 - DAPI')
ax[1].imshow(images_FISH[10,:,:,1],cmap='Spectral_r')
ax[1].set(title= 'Ch1 - FISH vs MS2  reporter gene' )
ax[2].imshow(images_FISH[10,:,:,2],cmap='Spectral_r')
ax[2].set(title= 'Ch1 - FISH vs GAPDH' )
plt.show()

# From the image we could determine 
channel_nucleus = 0
channel_cytosol = 2

# Creating  maximum projections for channel 0 and channel 2.
max_z_projection_nucleus = np.max(images_FISH[:,:,:,channel_nucleus], axis=0)
max_z_projection_cytosol = np.max(images_FISH[:,:,:,channel_nucleus], axis=0)

# Notice that this projection reduced the dimenssions to
print('Dimensions in tensor with nucleus image : ' , max_z_projection_nucleus.shape)
print('Dimensions in tensor with cytosol image : ' , max_z_projection_cytosol.shape)

# Using cellpose

# NUCLEUS
model = models.Cellpose(gpu=use_GPU, model_type='nuclei') # model_type='cyto' or model_type='nuclei'
masks_nuc, _, _, _ = model.eval(max_z_projection_nucleus, diameter=100, flow_threshold=None, channels=[0,0],  min_size=1000,net_avg=True, augment=True)
plt.title('Nucleus segmentation')
plt.imshow(masks_nuc,cmap='Greys_r')
plt.show()

# CYTOSOL
model = models.Cellpose(gpu=use_GPU, model_type='cyto2') # model_type='cyto', 'cyto2' or model_type='nuclei'
masks_cyto, _, _, _ = model.eval(max_z_projection_cytosol, diameter=200, flow_threshold=None, channels=[0,2],  min_size=1000,net_avg=True, augment=True)
plt.imshow(masks_cyto,cmap='Greys_r')
plt.title('Cytosol segmentation')
plt.show()



* Q5.  Calculate the average area for the cytsol (not including the nucleus) for all cells in the FISH image.

In [ ]:
# Add your code with your response here:





## Advanced  questions

* Q6. Calculate the center of mass of each cell in the FISH image.

In [ ]:
# Add your code with your response here:
number_masks = np.max(masks_cyto)
list_center_mass =[]
print('number cellls in image : ', number_masks)
for i in range (1,number_masks+1):
  # Calculaitng  the center of mass for a selected cell.
  selected_mask_cyto = np.where(masks_cyto==i,1,0) # Selecting only one mask.    
  y_indexes = np.nonzero(selected_mask_cyto)[0] # Detecting the indexes for all x axis
  x_indexes = np.nonzero(selected_mask_cyto)[1] # Detecting the indexes for all y axis
  center_mass_x = int ( np.sum(x_indexes) / np.sum(selected_mask_cyto) )
  center_mass_y = int( np.sum(y_indexes) / np.sum(selected_mask_cyto) )
  list_center_mass.append((center_mass_x,center_mass_y ) )
  del y_indexes, x_indexes, selected_mask_cyto, center_mass_x, center_mass_y

# Plotting the selected cell and its center of mass
plt.figure(figsize=(7,7))
plt.imshow(masks_cyto,cmap='Greys_r')
for i in range(len(list_center_mass)):
  plt.scatter(list_center_mass[i][0],list_center_mass[i][1],s=30)
plt.show()

* Q7. Make a widget that allows you to select different segmentation thresholds and apply the watershed algorithm to the image.

In [ ]:
# Add your code with your response here:





# References

*  Image downloaded from https://figshare.com from publication: "Forero-Quintero, Linda, William Raymond, Tetsuya Handa, Matthew Saxton, Tatsuya Morisaki, Hiroshi Kimura, Edouard Bertrand, Brian Munsky, and Timothy Stasevich. "Live-cell imaging reveals the spatiotemporal organization of endogenous RNA polymerase II phosphorylation at a single gene." (2020)."

* "Fox, Z.R., Fletcher, S., Fraisse, A., Aditya, C., Sosa-Carrillo, S., Gilles, S., Bertaux, F., Ruess, J. and Batt, G., 2021. MicroMator: Open and Flexible Software for Reactive Microscopy. bioRxiv. (2021)"